In [1]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('../Data/dataset_1000_minimum_reviews.csv', index_col='Unnamed: 0')
df = data[['time', 'movie_id', 'user_id', 'binary_rating']]
df.head()

,time,movie_id,user_id,binary_rating
0,0,1961,1107,1
1,1,1259,1545,0
2,2,595,3304,1
3,3,260,5448,0
4,4,1784,3829,0


In [6]:
def policy_evaluator_moss(dataframe, alpha):
    # We stock the payoffs in a list
    payoffs = []
    # We pull each arm once to initialize history
    history = dataframe.groupby('movie_id').first()
    arms = dataframe['movie_id'].unique()
    n_arms = len(arms)
    history['movie_id'] = history.index
    # We drop the rows associated to the initial pull
    rows_to_drop = history['time']
    history['time'] = 0
    dataframe_copy = dataframe.copy()
    dataframe_copy.drop(rows_to_drop, inplace=True)
    dataframe_copy.reset_index(drop=True, inplace=True)
    dataframe_copy['time'] = dataframe_copy.index
    # We create a dict containing all the movies ID and their corresponding number of pulls
    s = {}
    for movie in arms:
        s[movie] = 1
    for t in range(1, len(dataframe_copy) + 1):
        # We get t-th row of our dataframe
        t_event = dataframe_copy[t-1:t]
        # We get the recommendation of our algorithm
        objective_function = {}
        for k in arms:
            objective_function[k] = history[history['movie_id'] == k]['binary_rating'].mean() + np.sqrt( ( (1 + alpha) / 2) * max(np.log( t / (n_arms * s[k] ) ), 0) / s[k] )
        
        arm_chosen = max(objective_function, key=objective_function.get)

        s[arm_chosen] += 1

        if arm_chosen == t_event['movie_id'].iloc[0]:
            history.loc[len(history)] = t_event.iloc[0].to_list()
            payoffs.append(t_event['binary_rating'].iloc[0])

    return payoffs

In [7]:
payoffs = policy_evaluator_moss(df, 1.35)